# Data Recreation

---

>   config info below

## Overview

Now that we have the specific requested images, I just want to reassemble the data and write it out to a _clean_ `.csv` file.

## Setup


### Working Directory

This just helps with using local imports from the larger project to the notebook.

In [1]:
cd ../

/Users/chrismessier/work/behaviorally


### Imports

In [2]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
from numpy import random as rng
import matplotlib as mpl
from matplotlib import pyplot as plt
from google.protobuf.struct_pb2 import Struct
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_pb2, status_code_pb2

import processors
import tools

#### Config

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# for replicability
rng.seed(42)
# rng.seed(304)

In [5]:
%matplotlib inline

mpl.rcParams['figure.figsize'] = (12, 9)

sns.set(
    style='darkgrid'
)

In [6]:
from config import KEY_METRICS,\
    KEY_CONDITIONALS,\
    TO_NORMALIZE,\
    ONS_ANALYSIS_JOB_NUMBERS,\
    IMAGES_OF_INTEREST

In [7]:
N = 51  # sample size for images, keep LOW for dev

#### Functions

# Work

## Plan

Import the images of interest from Behaviorally. 
Load Tony's prediction results.
Pare down to the results to the items of interest.
Match with the IRI info, leaving out any cruft that was in previous iterations due to my matching attempts (aka upc).

Use this as a demonstration of setting data delivery standards!

In [8]:
ONS_OUTPUT = "/Users/chrismessier/work/behaviorally/data/ONS_output_Tony_Round 1.xlsx"

In [9]:
ons = pd.read_excel(ONS_OUTPUT)

In [10]:
ons.sample(3)

,Image Name,Image ID,Raw ONS Line and Pack
2313,AD090_PACK3C.jpg,PACK3C,82.0
315,AB474_LINE13.jpg,LINE13,82.0
2328,AD106_PACK1.jpg,PACK1,71.0


## Images of Interest

In [11]:
from config import IMAGES_OF_INTEREST

In [12]:
total_ioi = len(IMAGES_OF_INTEREST)

print(f"Total images of interest: {total_ioi}", end="\n\n")

Total images of interest: 51



In [13]:
ons_ = ons[ons['Image Name'].isin(IMAGES_OF_INTEREST)].copy()

In [14]:
nrows, ncols = ons_.shape

In [15]:
missing_images = [x for x in IMAGES_OF_INTEREST if x not in ons['Image Name'].values]

In [16]:
missing_images

['AD692_LINE1.png', 'L2115_LINE1.jpg', 'AD615_LINE1.jpg']

## Product Info

I'm going to just use the _most_ recent data they've shared with us.

In [17]:
PRODUCT_INFO = "/Users/chrismessier/work/behaviorally/data/[FINALIZED] IRI Data Product List_3.21.22.xlsx"

In [18]:
products = pd.read_excel(PRODUCT_INFO, sheet_name='50 products')
products.dropna(how='all', axis=0, inplace=True)
products.drop(columns=['SKU #'], inplace=True)  # I'm keeping jobnumber as that's used by them internally, and the rest for data richness.


Their team has been using the column 'Existing Images' as the ground truth.

In [19]:
products.sample(3)

,JobNumber,Category,Product description,Variant,UPC,Existing Images,Scraped Online Images
146,AB128.00,H/B - Beauty - Hair Care - Shampoo & Conditioner,Vamousse Lice Treatment Complete Kit,"Regular, Complete Kit, Shampoo",358962190407,NaN,NaN
169,L2402.00,H/B - Health - Feminine Care/Incontinence,One by Poise Panty Liners (2-in-1 Period & Bla...,"2 in 1 liners 50 ct, 2 in 1 pads 22 ct, 2 in 1...",36000534481,NaN,NaN
155,AD688.00,H/B - Beauty - Skin Care,"Banana Boat Sport Ultra Sunscreen Lotion 8 Oz,...","sport ulta 50, kids sport 50, baby mineral 50",79656049725,NaN,NaN


In [20]:
products_ = products[products['Existing Images'].isin(IMAGES_OF_INTEREST)]

In [32]:
products_.shape

(50, 7)

In [40]:
upcs = products_['UPC'].astype(str).tolist()

In [41]:
upcs

['71990281519',
 '3024480008797',
 '613008748333',
 '87692006602',
 '8811051056',
 '12000012754',
 '48500022382',
 '43000043011',
 '786162002815',
 '881334009880',
 '762111453983',
 '82592720641',
 '31200034694',
 '087684001073',
 '49000000443',
 '12000206351',
 '12000001291',
 '70462035964',
 '34000002290',
 '7020054430',
 '2100030047',
 '20735110225',
 '21000658831',
 '4112907712',
 '5210004228',
 '44000050986',
 '2225266004',
 '028400598293',
 '1162002305',
 '22200954419',
 '41167066515',
 '841058013211',
 '761596818584',
 '11111124257',
 '36000534771',
 '35000985026',
 '300672039248',
 '885898000420',
 '46500017926',
 '35103131',
 '3320009994',
 '11027237',
 '13700824258',
 '35000973351',
 '24200043872',
 '36000494358',
 '300026995961',
 '19014610907',
 '148967228',
 '32700155858']

In [35]:
img_upc_map = {}

for i, row in products_.iterrows():
    img_upc_map[row['Existing Images']] = row['UPC']


## Report Data

In [22]:
SALES_DATA = "/Users/chrismessier/work/behaviorally/data/Behaviorally - Report 4.18.22[8].xlsx"

Sales report data for the products here.


In [24]:
d = pd.read_excel(SALES_DATA, sheet_name=None)

In [25]:
report_dates = tools.infer_report_dates(range(1, 157))

In [26]:
sales = processors.behaviorally_reports(SALES_DATA)

In [27]:
sales['report_dates'] = sales['REPORT'].apply(lambda i: report_dates[int(i)-1])

In [29]:
sales.sample(1)

,Product,Dollar Sales,Dollar Sales % Change vs YA,Unit Sales,Unit Sales % Change vs YA,Unit Share of Category,Unit Share of Category Year Ago,Unit Share of SubCategory,Unit Share of SubCategory Year Ago,Price per Unit,Price per Unit % Change vs YA,UPC 10 digit,Category Name,Sub-Category Name,Brand Name,REPORT,report_dates
13,SOUR PATCH KIDS ASSORTED SOUR CHEWY CANDY PIEC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0462035964,NON-CHOCOLATE CANDY,NON CHOCOLATE CHEWY CANDY,SOUR PATCH KIDS,30,2019-10-27


Obviously the upc mapping is just unavoidable.